# 딥러닝 모델 서빙과 병렬처리

## 병렬처리
- [참조](/notebooks/머신러닝엔지니어/병렬처리/병렬처리.ipynb)
- 프로세스는 컴퓨터에서 연속적으로 실행되고 있는 컴퓨터 프로그램
- 쓰레드는 프로세스 내에서 실행되는 흐름의 단위
- 병렬처리는 여러 쓰레드를 활용하는 방법이 있고, 여러 개의 프로세스를 활용하는 방법이 있으며 각각 **멀티 쓰레드**, **멀티 프로세싱**이라고 부름.
- 멀티 프로세싱은 여러 개의 프로세스가 별도로 실행되고 각 프로세스가 별개의 메모리를 차지하고 있으며, 멀티 쓰레드는 하나의 프로세스 내에서 메모리를 공유해서 사용.


### python의 GIL
- [참조](https://changminkimserver29672.iptime.org:62911/notebooks/머신러닝엔지니어/병렬처리/파이썬_GIL.ipynb#Race-condition-문제)
- Global Interpreter Lock
- 파이썬 인터프리터가 한 스레드만 하나의 바이트코드를 실행 시킬 수 있도록 해주는 **Mutex**
- 하나의 스레드에 모든 자원을 허락하고 그 후에는 Lock을 걸어 다른 스레드는 실행할 수 없게 막아버리는 것
- 즉 파이썬 쓰레드는 한번에 하나 밖에 동작을 못 함.

In [1]:
import random
import threading
import time

In [21]:
def working():
    max([random.random() for i in range(20000000)])

In [22]:
# 1 Thread
s_time = time.time()
working()
working()
e_time = time.time()
print(f'{e_time - s_time:.5f}')

3.20173


In [23]:
# 2 Threads
s_time = time.time()
threads = []
for i in range(2):
    threads.append(threading.Thread(target=working))
    threads[-1].start()

for t in threads:
    t.join()

e_time = time.time()
print(f'{e_time - s_time:.5f}')


3.29190


## Tensorflow 와 Pytorch의 병렬처리
Tensorflow 나 Pytorch 같은 경우 코어 내부에 Python이 아닌 다른 언어가 동작하고 있어서 GIL의 영향을 받지 않고, inference를 수행하는 동안 **멀티 쓰레드**가 동작할 수 있다.

그래서 Flask를 활용하여 서빙할 때, 실행 시에 threaded=True를 사용하면, 요청이 동시에 들어와도 수행이 가능.

하지만 pytorch의 경우에는 현재 멀티 쓰레드를 사용하면 내부적으로 변수가 꼬이는 현상이 발생하므로 사용하시면 안된다.

Pytorch의 경우 **멀티 프로세싱**을 활용하면 되는데, 멀티 프로세싱은 여러 개의 프로세스를 실행시키는 방식으로, 각 프로세스가 별도로 동작하므로 pytorch에서 아무 문제가 없이 사용가능.

사실, tensorflow의 경우에도 멀티 쓰레드로 처리하는 것 보다, 멀티 프로세싱으로 처리하는 것이 더 빠르다.

멀티 프로세스를 사용한다고 하면 인퍼런스를 하는 서버를 여러 개 띄워두고, 요청이 들어올 때는 잘 중재하여 각 인퍼런스 서버에 나눠서 일을 시키도록 하는 모습

### Flask를 이용한 멀티 프로세스로 딥러닝 모델 서빙.
- Gunicorn같은 미들웨어를 활용해서 멀티 프로세스 구현.
- pip install gunicorn
- gunicorn flask서버파일:app --bind=0.0.0.0:포트 -w 워커개수

### Server 코드

In [1]:
import numpy as np
import tensorflow as tf
from flask import Flask, request

In [2]:
load = tf.saved_model.load('../model/mnist/1/')

In [3]:
load_inference = load.signatures['serving_default']

In [4]:
app = Flask(__name__)

@app.route('/inference',methods=['POST'])
def inference():
    global data
    data = request.json
    image = data['images']
    pre_image = tf.reshape(tf.constant(image,dtype=tf.float32)/255.0,(1,28,28,1))
    result = load_inference(pre_image)
    print(result)
    print(str(np.argmax(result['dense'].numpy())))
    return str(np.argmax(result['dense'].numpy()))

if __name__ == '__main__':
    app.run(host='0.0.0.0',port='41253')

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:41253
 * Running on http://172.17.0.4:41253
Press CTRL+C to quit
192.168.0.1 - - [23/Oct/2022 13:50:23] "POST /inference HTTP/1.1" 200 -


{'dense': <tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[3.5300819e-04, 6.5266178e-03, 7.9976125e-03, 2.4240771e-03,
        4.7246707e-03, 3.5549065e-03, 1.8990660e-04, 9.6926939e-01,
        2.2097756e-03, 2.7500829e-03]], dtype=float32)>}
7


192.168.0.1 - - [23/Oct/2022 13:50:46] "POST /inference HTTP/1.1" 200 -


{'dense': <tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[3.5300819e-04, 6.5266178e-03, 7.9976125e-03, 2.4240771e-03,
        4.7246707e-03, 3.5549065e-03, 1.8990660e-04, 9.6926939e-01,
        2.2097756e-03, 2.7500829e-03]], dtype=float32)>}
7


192.168.0.1 - - [23/Oct/2022 13:50:57] "POST /inference HTTP/1.1" 200 -


{'dense': <tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[3.5300819e-04, 6.5266178e-03, 7.9976125e-03, 2.4240771e-03,
        4.7246707e-03, 3.5549065e-03, 1.8990660e-04, 9.6926939e-01,
        2.2097756e-03, 2.7500829e-03]], dtype=float32)>}
7


192.168.0.1 - - [23/Oct/2022 13:51:07] "POST /inference HTTP/1.1" 200 -


{'dense': <tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[8.4653095e-04, 2.8771220e-03, 1.7690003e-02, 4.4492311e-03,
        4.6465406e-03, 5.0320975e-03, 9.1941254e-03, 6.5094652e-04,
        9.5343238e-01, 1.1809780e-03]], dtype=float32)>}
8


### gunciron 실행
- worker는 4개로 지정.(4개의 프로세스에서 나눠서 병렬로 처리)
- --daemon 옵션을 추가하면 백그라운드에서 실행가능.

In [ ]:
gunicorn flask_server:app --bind=0.0.0.0:41253 -w 4

In [1]:
gunicorn

NameError: name 'gunicorn' is not defined